In [236]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select

## Scraping and Building the Washington State Elementary Schools Data

This notebook details the process I used to build the Washington State elementary schools test scores and demographics data set, that I then used to create the Student Independent Performance metric.

The data were taken from the website schooldigger.com, which has in various tables standardized test scores and demographic information for elementary, middle, and high schools across the United States. Their data is aggregated from the National Center for Education Statistics, U.S. Department of Education, the U.S. Census Bureau, the Washington State Department of Health and the Washington Office of Superindentent of Public Instruction.

### Test Score Table

First we'll start our Selenium web driver and navigate to the main table page for Washington elementary schools, which contain test score data.

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get("https://www.schooldigger.com/go/WA/schoolrank.aspx")

Next we'll grab the column headers, and then the first page of results.

In [247]:
html = BeautifulSoup(driver.page_source, 'html.parser')

In [248]:
tables = html.find_all('tr')

In [249]:
table_columns = {}
column_index = 0

for header in tables[1].find_all("th"):
    column = " ".join(list(header.stripped_strings))
    table_columns[column_index] = column
    column_index += 1
    
html = BeautifulSoup(driver.page_source, 'html.parser')
tables = html.find_all('tr')

data = []

for table_row in tables[2:]:
    row = {}
    column_index = 0
    
    for table_data in table_row.find_all("td"):
        row[column_index] = "".join(list(table_data.strings)).replace('\n', '')
        column_index += 1
    
    data.append(row)

We'll need to click the pagination buttons to navigate through the remaining pages of the table, with a little index finessing to get the data at the beginning and end.

In [257]:
p_index = 1 

In [478]:
while(p_index < 107):
    if p_index < 4:
        button_index = p_index
    else if p_index > 104:
        button_index = p_index - 100
    else:
        button_index 
    
    pagination = driver.find_element_by_class_name("pagination")
    pagination.find_elements_by_tag_name("a")[button_index].click()

    p_index += 1
    
    html = BeautifulSoup(driver.page_source, 'html.parser')
    tables = html.find_all('tr')

    for table_row in tables[2:]:
        row = {}
        column_index = 0
    
        for table_data in table_row.find_all("td"):
            row[column_index] = "".join(list(table_data.strings)).replace('\n', '')
            column_index += 1
    
        data.append(row)

Let's store this table in a pandas data frame for now.

In [479]:
test_scores = pd.DataFrame(data)
test_scores.rename(columns = table_columns, inplace = True)


### Student Demographic Data

Now to scrape the student demographic data, we'll need to navigate our Selenium driver to that table.

In [ ]:
pagination = driver.find_element_by_class_name("pagination")
pagination.find_elements_by_tag_name("a")[0].click()

column_select = driver.find_element_by_class_name("btn-group")
column_select = driver.find_element_by_class_name("dropdown-menu")
cs = column_select.find_elements_by_tag_name("a")
cs[2].click()


Now we'll perform the same procedure as before, first grabbing the table columns and first few rows of data, before navigating through each page for the subsequent data.

In [ ]:
html = BeautifulSoup(driver.page_source, 'html.parser')
tables = html.find_all('tr')

In [ ]:
table_columns = {}
column_index = 0

for header in tables[1].find_all("th"):
    column = " ".join(list(header.stripped_strings))
    table_columns[column_index] = column
    column_index += 1
    
html = BeautifulSoup(driver.page_source, 'html.parser')
tables = html.find_all('tr')

data = []

for table_row in tables[2:]:
    row = {}
    column_index = 0
    
    for table_data in table_row.find_all("td"):
        row[column_index] = "".join(list(table_data.strings)).replace('\n', '')
        column_index += 1
    
    data.append(row)

And now to navigate subsequent pages.

In [ ]:
p_index = 1

In [ ]:
while(p_index < 107):
    if p_index < 4:
        button_index = p_index
    else if p_index > 104:
        button_index = p_index - 100
    else:
        button_index 
    
    pagination = driver.find_element_by_class_name("pagination")
    pagination.find_elements_by_tag_name("a")[button_index].click()

    p_index += 1
    
    html = BeautifulSoup(driver.page_source, 'html.parser')
    tables = html.find_all('tr')

    for table_row in tables[2:]:
        row = {}
        column_index = 0
    
        for table_data in table_row.find_all("td"):
            row[column_index] = "".join(list(table_data.strings)).replace('\n', '')
            column_index += 1
    
        data.append(row)

Last, we'll store the data in a data frame.

In [481]:
students = pd.DataFrame(data)
students.rename(columns = table_columns, inplace = True)

### Writing to file

Now that our data is in data frames, all that's left for us to do is join the tables and write to a csv file for storage.

In [488]:
combined = test_scores.join(students.set_index('Rank (of 1062 )'), on = 'Rank (of 1062 )',
                         lsuffix = "_a", rsuffix = "_b")

In [491]:
combined.to_csv("elementary_schools.csv", index = False)